In [1]:
pip install textstat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch

In [3]:

pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from tqdm import tqdm
import collections
import pickle
import re
import math
import textstat
import spacy
import torch
import torch.nn as nn
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ryanje/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ryanje/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ryanje/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ryanje/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
train_df = pd.read_csv('WikiLarge_Train.csv')
test_df = pd.read_csv('WikiLarge_Test.csv')

In [2]:

features_aoa = pd.read_csv('AoA_51715_words.csv',encoding='Windows-1252')
features_concrete = pd.read_csv('Concreteness_ratings_Brysbaert_et_al_BRM.txt',encoding='utf-8', delimiter = '\t')
features_dale_chall = pd.read_csv('dale_chall.txt',encoding='utf-8',delimiter='\t')

In [11]:
# Initialize the stemmer
stemmer = PorterStemmer()
# Initialize the lemmatizer and load Spacy's English model
lemmatizer = WordNetLemmatizer()
nlp = spacy.load("en_core_web_sm")

# Helper functions

#regex to clean the original text
pattern = re.compile(r'\-lrb\-|\-rrb\-|\b\W\b|\n|_|\s+')
def clean_text(sentence):
    # Lowercasing
    sentence = sentence.lower()
    # Applying compiled regular expression pattern
    sentence = pattern.sub(' ', sentence)
    # Stripping leading and trailing spaces
    sentence = sentence.strip()
    
    return sentence

# Lemmatize the tokens
def text_lemmatizer(token_text):
    return [lemmatizer.lemmatize(word) for word in token_text]

# dependency counts take way too long to create for minimal value
# # Adding dependency parsing counts
# def get_dependency_counts(text):
#     doc = nlp(text)
#     return Counter(tok.dep_ for tok in doc)

# def add_dependency_counts(dataframe):
#     dep_counts = dataframe['clean_text'].apply(get_dependency_counts)
#     unique_deps = set(dep for counts in dep_counts for dep in counts.keys())
#     for dep in unique_deps:
#         dataframe[f'dep_{dep}_count'] = dep_counts.apply(lambda x: x.get(dep, 0))

#add Parts of Speech counts, limited to some common ones as it takes a long time to run
def add_pos_counts(dataframe):
    pos_counts = dataframe['tokens'].apply(lambda tokens: Counter(tag for word, tag in nltk.pos_tag(tokens)))
    for tag in ['NN', 'VB', 'JJ', 'RB']:
        dataframe[f'pos_{tag}_count'] = pos_counts.apply(lambda x: x.get(tag, 0))

# Calculating entropy of sentence as a measure of how many unique words
def lexical_diversity(text):
    return -sum(freq/len(text) * math.log(freq/len(text), 2) for freq in Counter(text).values())

# Function to sum how many words in a sentence have more than one meaning
def count_multiple_meanings(tokens):
    return sum(len(wordnet.synsets(word)) > 1 for word in tokens)

# Function to calculate SMOG Index
def calculate_smog_index(text):
    # Using textstat library to calculate SMOG Index
    return smog_index(text)

# Function to calculate Type-Token Ratio (TTR)
def calculate_type_token_ratio(tokens):
    # Number of unique words
    types = set(tokens)
    # Type-Token Ratio
    return len(types) / len(tokens) if len(tokens) > 0 else 0

# Function to bin different readability tests into easy, average, or difficult texts
def calculate_readability_features(dataframe):
    
    # Add logic for updating categories based on 'clean_text'
    dataframe.loc[dataframe['clean_text'].isnull() | (dataframe['clean_text'] == 0), 'flesch_reading_ease_category'] = 2
    dataframe.loc[dataframe['clean_text'].isnull() | (dataframe['clean_text'] == 0), 'gunning_fog_category'] = 1
    dataframe.loc[dataframe['clean_text'].isnull() | (dataframe['clean_text'] == 0), 'smog_index_category'] = 2    
    
    # Flesch Reading Ease, higher score is easier to read
    # Categories: 0 = Easy, 1 = Average, 2 = Difficult
    dataframe['flesch_reading_ease'] = dataframe['clean_text'].apply(textstat.flesch_reading_ease)
    dataframe['flesch_reading_ease_category'] = dataframe['flesch_reading_ease'].apply(lambda x: 0 if x > 70 else (2 if x < 60 else 1))
    dataframe.drop(columns=['flesch_reading_ease'], inplace=True)

    # Gunning Fog is age a reader should be to understand a sentence
    # Categories: 0 = Easy, 1 = Average, 2 = Difficult
    dataframe['gunning_fog'] = dataframe['clean_text'].apply(textstat.gunning_fog)
    dataframe['gunning_fog_category'] = dataframe['gunning_fog'].apply(lambda x: 0 if x < 10 else (2 if x > 15 else 1))
    dataframe.drop(columns=['gunning_fog'], inplace=True)

    # Smog Index, similar to Gunning Fog
    # Categories: 0 = Easy, 1 = Average, 2 = Difficult 
    dataframe['smog_index'] = dataframe['clean_text'].apply(textstat.smog_index)
    dataframe['smog_index_category'] = dataframe['smog_index'].apply(lambda x: 0 if x < 10 else (2 if x > 15 else 1))
    dataframe.drop(columns=['smog_index'], inplace=True)

# Helper function for calculate_aoa_dalechall_features 
def calculate_additional_features(dataframe, average_aoa, dale_chall_lemmatized):
    lemma_texts = dataframe['lemmatized_text'].str.split()
    dale_chall_counts = lemma_texts.apply(lambda words: sum(word in dale_chall_lemmatized for word in words))
    high_aoa_counts = lemma_texts.apply(lambda words: sum((word in average_aoa) and (average_aoa[word] > 9) for word in words))
    low_aoa_counts = lemma_texts.apply(lambda words: sum((word in average_aoa) and (average_aoa[word] <= 9) for word in words))

    return dale_chall_counts, high_aoa_counts, low_aoa_counts
                                                                       
                                                                       
#function to calculate the counts of dale chall words and binned AoA words, lemmatized
#AoA ages are rounded and averaged as sum words are the same after lemmatizing
def calculate_aoa_dalechall_features(dataframe, features_aoa, features_dale_chall):
    # AoA DaleChall features using lemmatized words
    features_aoa.dropna(subset=['Word', 'AoA_Kup_lem'], inplace=True)
    aoa_lemmatized = text_lemmatizer(features_aoa['Word'])
    lemmatized_word_counts = {word: {'sum': 0, 'count': 0} for word in aoa_lemmatized}
    for idx, (word, aoa) in enumerate(zip(aoa_lemmatized, features_aoa['AoA_Kup_lem'])):
        lemmatized_word_counts[word]['sum'] += aoa
        lemmatized_word_counts[word]['count'] += 1
    average_aoa = {word: round(info['sum'] / info['count']) for word, info in lemmatized_word_counts.items()}
    dale_chall_lemmatized = set(text_lemmatizer(features_dale_chall['a']))
    
    dale_chall_counts, high_aoa_counts, low_aoa_counts = calculate_additional_features(dataframe, average_aoa, dale_chall_lemmatized)
    dataframe['dale_chall'] = dale_chall_counts
    dataframe['high_aoa'] = high_aoa_counts
    dataframe['low_aoa'] = low_aoa_counts

# Function to calculate features from concretness dataset on lemmatized words
def calculate_concreteness_features(dataframe, features_concrete):
    # Concreteness features using lemmatized words
    features_concrete['lemmatized_word'] = features_concrete['Word'].apply(lambda x: lemmatizer.lemmatize(x) if isinstance(x, str) else x)
    grouped_concreteness = features_concrete.groupby('lemmatized_word').agg({'Conc.M': 'mean', 'SUBTLEX': 'mean'}).reset_index()
    # Zipping features together to increase performance                                                                   
    concreteness_dict = dict(zip(grouped_concreteness['lemmatized_word'], grouped_concreteness['Conc.M']))
    subtlex_us_dict = dict(zip(grouped_concreteness['lemmatized_word'], grouped_concreteness['SUBTLEX']))
    
    # Using lemmatized text in the calculation of sentence features and using numpy arrays for faster calc
    num_sentences = len(dataframe['lemmatized_text'])
    concrete_count = np.zeros(num_sentences, dtype=int)
    non_concrete_count = np.zeros(num_sentences, dtype=int)
    mean_concreteness = np.zeros(num_sentences)
    mean_subtlex_us_frequency = np.zeros(num_sentences)

    for i, text in enumerate(dataframe['lemmatized_text']):
        words = text.split()
        concreteness_scores = [concreteness_dict[word] for word in words if word in concreteness_dict]
        subtlex_us_frequencies = [subtlex_us_dict[word] for word in words if word in subtlex_us_dict]

        if concreteness_scores:
            mean_concreteness[i] = np.mean(concreteness_scores)
            concrete_count[i] = sum(1 for score in concreteness_scores if score > 3)
            non_concrete_count[i] = len(concreteness_scores) - concrete_count[i]

        if subtlex_us_frequencies:
            mean_subtlex_us_frequency[i] = np.mean(subtlex_us_frequencies)
            
    # Store features in the dataframe
    dataframe['concrete_count'] = concrete_count
    dataframe['non_concrete_count'] = non_concrete_count
    dataframe['mean_concreteness'] = mean_concreteness
    dataframe['mean_subtlex_us_frequency'] = mean_subtlex_us_frequency




# Main function to extract features from a DataFrame
def extract_features(dataframe, features_aoa, features_dale_chall, features_concrete):
    
    # Feature for Wikipedia markup LRB or RRB
    dataframe['contains_wiki_markup'] = dataframe['original_text'].str.contains(r'-LRB-|-RRB-', regex=True).astype(int)
    dataframe['is_numeric'] = dataframe['original_text'].apply(lambda x: ''.join(filter(str.isdigit, x)).isdigit()).astype(int)
    # Feature for sentences that are only a punctuation mark
    dataframe['only_punctuation'] = dataframe['original_text'].str.match(r'^\W+$').astype(int)
    print('done checking nums/punct/foreign words')
    # Clean and tokenize text
    dataframe['clean_text'] = dataframe['original_text'].apply(clean_text)
    dataframe['tokens'] = dataframe['clean_text'].apply(lambda x: word_tokenize(x))
    dataframe['lemmatized_text'] = dataframe['tokens'].apply(lambda x: " ".join(text_lemmatizer(x)))
    dataframe['only_markup'] = np.where(dataframe['clean_text'].isnull(), 1, 0)
    print('done lemmatizing')
   
    # Word and character counts
    dataframe['num_words'] = dataframe['tokens'].str.len()
    dataframe['num_characters'] = dataframe['clean_text'].str.len()
    dataframe['num_unique_words'] = dataframe['tokens'].apply(lambda x: len(set(x)))
    dataframe['ratio_unique_total_words'] = dataframe['num_unique_words'] / dataframe['num_words']
    dataframe['num_long_words'] = dataframe['tokens'].apply(lambda x: sum(1 for word in x if len(word) > 6))
    dataframe['syllable_count'] = dataframe['clean_text'].apply(textstat.syllable_count)
    dataframe['avg_syllables_per_word'] = dataframe['syllable_count'] / dataframe['num_words']
    dataframe['lexical_diversity'] = dataframe['clean_text'].apply(lexical_diversity) 
    dataframe['type_toke_ration'] = dataframe['tokens'].apply(calculate_type_token_ratio)
    dataframe['multiple_meanings_count'] = dataframe['tokens'].apply(count_multiple_meanings)
    

    print('done adding counts')
#     #Depencdency Counts
#     add_dependency_counts(dataframe)
#     print('dependency counts')
    # Parts of Speech Counts
    add_pos_counts(dataframe)
    print('pos counts')
    # Readability features
    calculate_readability_features(dataframe)
    print('readability counts')
    # AoA DaleChall features
    calculate_aoa_dalechall_features(dataframe, features_aoa, features_dale_chall)
    print('aoa and dalechall')
    # Concreteness features
    calculate_concreteness_features(dataframe, features_concrete)    
    #some text is completely stripped, resulting in null calculations 
    dataframe.fillna(0, inplace=True)

    return dataframe



In [5]:
train_df = extract_features(train_df,features_aoa, features_dale_chall, features_concrete)

done checking nums/punct/foreign words
done lemmatizing
done adding counts
pos counts
readability counts
aoa and dalechall


In [12]:
test_df = extract_features(test_df, features_aoa, features_dale_chall, features_concrete)

done checking nums/punct/foreign words
done lemmatizing
done adding counts
pos counts
readability counts
aoa and dalechall


In [12]:
def print_rows_with_nulls(dataframe):
    # Check for rows with null values
    rows_with_nulls = dataframe[dataframe.isna().any(axis=1)]
    
    # If there are any rows with null values, print them
    if not rows_with_nulls.empty:
        print("Rows with null values:")
        print(rows_with_nulls)
    else:
        print("No rows with null values found.")

In [13]:
print_rows_with_nulls(train_df)

No rows with null values found.


In [14]:
print_rows_with_nulls(test_df.drop('label',axis=1))

No rows with null values found.


In [15]:
with open('train_df.pickle', 'wb') as f:
    pickle.dump(train_df, f)

In [2]:
#load pickled data
with open('train_df.pickle', 'rb') as f:
    train_df = pickle.load(f)

In [16]:
with open('test_df.pickle', 'wb') as f:
    pickle.dump(test_df, f)

In [3]:
#load pickled data
with open('test_df.pickle', 'rb') as f:
    test_df = pickle.load(f)

In [13]:
# copy dataframes and remove all text for classifiers that don't use text
def df_copy_nontext(df):
    X = df.copy()
    X = X.drop(['original_text','clean_text','tokens','lemmatized_text'],axis=1)
    if 'id' in df.columns:
        X = X.drop('id',axis=1)
    y = X.pop('label')
    return X, y


X, y = df_copy_nontext(train_df)
X_test, _ = df_copy_nontext(test_df)

In [18]:
# Split the data into training and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
#X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns)
#X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [21]:
# Create a DummyClassifier with a 'most_frequent' strategy
dummy = DummyClassifier(strategy='most_frequent')

# Train the classifier on the training data
dummy.fit(X_train_scaled_df, y_train)

# Make predictions on the test data
y_pred = dummy.predict(X_val_scaled_df)

# Calculate the accuracy of the classifier
accuracy_score = accuracy_score(y_val, y_pred)
print("F1:", accuracy_score)

F1: 0.49843240130526584


In [76]:
lr_clf = LogisticRegression(random_state=42, n_jobs=-1).fit(X_train_scaled_df,y_train)
preds = lr_clf.predict(X_val_scaled_df)
f1_score = f1_score(y_val, preds)
print(f1_score)

0.6593907905783213


In [77]:

# # Get feature names, assuming you are using DataFrame
# feature_names = X_train_scaled_df.columns

# # Get the coefficients from the trained model
# coefficients = lr_clf.coef_[0]

# # Combine feature names and coefficients into a DataFrame
# feature_importances = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

# # Sort the features by the absolute value of their coefficient
# feature_importances = feature_importances.reindex(feature_importances.coefficient.abs().sort_values(ascending=False).index)

# # Display the feature importances
# print(feature_importances)


In [99]:
rf_clf = RandomForestClassifier(random_state=42,n_estimators=100,max_depth=5, n_jobs=-1).fit(X_train_scaled_df, y_train)
preds = rf_clf.predict(X_val_scaled_df)
f1_scores = f1_score(y_val, preds)
print(f1_scores)

0.6684528631067356


In [23]:
# Refit the classifier on the full training and validation set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
lr_clf = LogisticRegression(random_state=42, n_jobs=-1).fit(X_scaled,y)

# Predict X_test
y_test_pred = lr_clf.predict(X_test_scaled)

# Create a DataFrame with 'id' and 'label' columns
results_df = pd.DataFrame({'id': test_df['id'], 'label': y_test_pred})

# Save the results to a CSV file
results_df.to_csv('lr_predictions.csv', index=False)

In [106]:
#refit if needed for full data on random forest
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
rf_clf = RandomForestClassifier(random_state=42,n_estimators=100,max_depth=7, n_jobs=-1).fit(X_scaled,y)
# Predict X_test
y_test_pred = rf_clf.predict(X_test_scaled)

# Create a DataFrame with 'id' and 'label' columns
results_df = pd.DataFrame({'id': test_df['id'], 'label': y_test_pred})

# Save the results to a CSV file
results_df.to_csv('rf_predictions.csv', index=False)

In [115]:
#max depth 10 and tol .001 did the best so far, trying deeper depth
gb_model = GradientBoostingClassifier(n_estimators = 200, max_depth = 10,
                                    random_state=42,verbose=1,n_iter_no_change=10,tol=0.001).fit(X_train_scaled_df,y_train)
preds = gb_model.predict(X_val_scaled_df)
f1_scores = f1_score(y_val, preds)
print(f1_scores)


      Iter       Train Loss   Remaining Time 
         1           1.3504            9.32m
         2           1.3203            9.30m
         3           1.2949            9.23m
         4           1.2737            9.21m
         5           1.2551            9.16m
         6           1.2391            9.12m
         7           1.2252            9.07m
         8           1.2129            9.03m
         9           1.2018            8.99m
        10           1.1920            8.94m
        20           1.1300            8.52m
        30           1.0948            8.07m
        40           1.0717            7.64m
        50           1.0554            7.21m
        60           1.0432            6.77m
        70           1.0322            6.32m
        80           1.0219            5.86m
        90           1.0109            5.38m
0.7052010690915794


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
gb_model = GradientBoostingClassifier(n_estimators = 200, max_depth = 15,
                                    random_state=42,verbose=1,n_iter_no_change=10,tol=0.001).fit(X_scaled,y)
# Predict X_test
y_test_pred = gb_model.predict(X_test_scaled)

# Create a DataFrame with 'id' and 'label' columns
results_df = pd.DataFrame({'id': test_df['id'], 'label': y_test_pred})

# Save the results to a CSV file
results_df.to_csv('gb_predictions.csv', index=False)

      Iter       Train Loss   Remaining Time 
         1           1.2825           42.27m
         2           1.1923           46.84m
         3           1.1124           49.74m
         4           1.0395           52.05m
         5           0.9763           53.35m
         6           0.9184           54.43m
         7           0.8662           55.12m
         8           0.8180           55.84m
         9           0.7745           56.33m
        10           0.7334           56.79m


In [4]:
#keep lemmatized sentence for text transformations
def df_copy_text(df):
    X = df.copy()
    X = X.drop(['original_text','clean_text','tokens'],axis=1)
    if 'id' in df.columns:
        X = X.drop('id',axis=1)
    y = X.pop('label')
    return X, y


X, y = df_copy_text(train_df)
X_test, _ = df_copy_text(test_df)

In [ ]:
# try combining tf-idf with other features, might need svd/pca

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [27]:
vectorizer = TfidfVectorizer(stop_words='english',min_df=60,max_df=0.8,strip_accents='unicode')
X_train_tf = vectorizer.fit_transform(X_train['lemmatized_text'])
X_val_tf = vectorizer.transform(X_val['lemmatized_text'])
# Scale the data
scaler_vec = StandardScaler(with_mean=False)  # use with_mean=False for sparse data

scaled_tfidf_train = scaler_vec.fit_transform(X_train_tf)
scaled_tfidf_val = scaler_vec.transform(X_val_tf)


 

In [28]:
lr_clf_vec = LogisticRegression(random_state=42, n_jobs=-1).fit(scaled_tfidf_train,y_train)
preds = lr_clf_vec.predict(scaled_tfidf_val)
acc_score = accuracy_score(y_val, preds)
print(acc_score)

0.676994689359524


<354252x8442 sparse matrix of type '<class 'numpy.float64'>'
	with 2929623 stored elements in Compressed Sparse Row format>

In [17]:
# n_components = 2  # Start with 2 components
# cumulative_explained_variance = 0
# while cumulative_explained_variance < 0.9:
#     svd = TruncatedSVD(n_components=n_components)
#     svd.fit(scaled_tfidf_train)
#     cumulative_explained_variance = sum(svd.explained_variance_ratio_)
#     n_components += 1

# # Decrement n_components by 1 because it was incremented one extra time in the loop
# n_components_90 = n_components - 1
# print(f"Number of components needed to retain 90% of variance: {n_components_90}")

# # Now reapply SVD with the found number of components
# svd_90 = TruncatedSVD(n_components=n_components_90)
# reduced_data_90_train = svd_90.fit_transform(scaled_tfidf_train)
# reduced_data_90_val = svd_90.transform(scaled_tfidf_val)

In [34]:
# #1000 catches .11 percent
# svd = TruncatedSVD(n_components=2000)
# svd.fit(scaled_tfidf_train)
# cumulative_explained_variance = sum(svd.explained_variance_ratio_)
# print(cumulative_explained_variance)

In [30]:
svd_90 = TruncatedSVD(n_components=100)
reduced_data_90_train = svd_90.fit_transform(scaled_tfidf_train)
reduced_data_90_val = svd_90.transform(scaled_tfidf_val)

In [31]:
#100 components got .6094
#1000 components got .6275
lr_clf_vec = LogisticRegression(random_state=42, n_jobs=-1).fit(reduced_data_90_train,y_train)
preds = lr_clf_vec.predict(reduced_data_90_val)
acc_score = accuracy_score(y_val, preds)
print(acc_score)

0.6122272698189264


In [ ]:
rf_clf_vec = RandomForestClassifer(random_state=42, max_depth = 10, n_jobs=-1).fit(reduced_data_90,y_train)
preds = rf_clf_vec.predict(reduced_data_90_val)
acc_score = accuracy_score(y_val, preds)
print(acc_score)

In [37]:

# gb_model_vec = GradientBoostingClassifier(n_estimators = 200, max_depth = 20,
#                                     random_state=42,verbose=1,n_iter_no_change=10,tol=0.001,learning_rate=0.15).fit(scaled_tfidf_train,y_train)
# preds = gb_model_vec.predict(scaled_tfidf_val)
# acc_score = accuracy_score(y_val, preds)
# print(acc_score)

In [20]:
X_train.head()

,contains_wiki_markup,is_numeric,only_punctuation,lemmatized_text,num_words,num_characters,num_unique_words,ratio_unique_total_words,num_long_words,syllable_count,...,gunning_fog_category,smog_index_category,dale_chall,high_aoa,low_aoa,concrete_count,non_concrete_count,mean_concreteness,mean_subtlex_us_frequency,only_markup
410396,0,0,0,it contains the east half of new guinea island...,26,132,21,0.807692,4,31,...,0,0,18,0,22,9,12,2.964524,288765.500000,0
158130,0,0,0,she wa even more vile and full of sin than aha...,30,132,24,0.800000,1,29,...,1,0,23,2,23,5,20,2.348600,353757.120000,0
47225,1,1,0,bar hill is a purpose built village with a pop...,28,136,25,0.892857,6,34,...,1,0,15,0,18,9,9,2.848889,284176.194444,0
322386,1,1,0,"however , the storm weakened again beause of a...",46,239,39,0.847826,10,55,...,1,0,16,1,22,9,12,2.726429,252962.000000,0
391668,1,1,0,formula one season sometimes referred to a the...,32,179,29,0.906250,6,45,...,1,0,20,4,23,5,18,2.177826,369295.260870,0


In [23]:
#with 1000 svd components and additional features, score goes to 0.6647
# Scale the additional features
scaler_additional = StandardScaler()
num_features_train = scaler_additional.fit_transform(X_train.drop('lemmatized_text',axis=1))
num_features_val = scaler_additional.transform(X_val.drop('lemmatized_text',axis=1))

# Concatenate SVD features with additional features
combined_features_train = np.hstack((reduced_data_90_train, num_features_train))
combined_features_val = np.hstack((reduced_data_90_val, num_features_val))

# Train a classifier (e.g., Random Forest)
classifier = LogisticRegression(random_state=42, n_jobs=-1)
classifier.fit(combined_features_train, y_train)  # Assuming y_train is your target for the training set

# Evaluate the classifier on the validation set
score = classifier.score(combined_features_val, y_val)  # Assuming y_val is your target for the validation set
print(f"Validation Score: {score}")


Validation Score: 0.6647418260925203


/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
# # Train a classifier (e.g., Random Forest)
# classifier = LogisticRegression(random_state=42,solver='saga', max_iter=1000,C=0.1, n_jobs=-1)
# classifier.fit(combined_features_train, y_train)  

# # Evaluate the classifier on the validation set
# score = classifier.score(combined_features_val, y_val)  
# print(f"Validation Score: {score}")

In [ ]:
rf_clf_vec = RandomForestClassifer(random_state=42, max_depth = 10, n_jobs=-1).fit(combined_features_train,y_train)
preds = rf_clf_vec.predict(combine_features_val)
acc_score = accuracy_score(y_val, preds)
print(acc_score)

In [ ]:
gb_model_vec = GradientBoostingClassifier(n_estimators = 300, max_depth = 15,
                                    random_state=42,verbose=1,n_iter_no_change=10,tol=0.001,n_jobs=-1).fit(combined_features_train,y_train)
score = gb_model_vec.score(combined_features_val, y_val) 
print(f"Validation Score: {score}")

      Iter       Train Loss   Remaining Time 


In [ ]:
import torch
from transformers import BertTokenizer, BertModel


# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode text to BERT embeddings
def encode_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Get the output from BERT model
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the embeddings (use the last hidden state)
    # The output is in shape (batch_size, sequence_length, hidden_size)
    # We can use mean pooling to get fixed size encoding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return embeddings

# Apply the function to the 'lemmatized_text' column
X['bert_embeddings'] = X['lemmatized_text'].apply(encode_text)
print('X dataframe done')
X_test['bert_embeddings'] = X_test['lemmatized_text'].apply(encode_text)

X.head()

In [ ]:
#add bert embeddings to main dataframes and then pickle again
train_df['bert_embeddings'] = X['bert_embeddings']
test_df['bert_embeddings'] = X_test['bert_embeddings']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Convert the embeddings to numpy array (df['bert_embeddings'] is a series of tensors)
embeddings = torch.stack(df['bert_embeddings'].tolist()).numpy()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Train a classifier
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)

# Evaluate the model
score = f1_score(y_test, predictions)
print(f'F1 Score: {score}')


In [ ]:
# Apply standard scaling to the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Convert the scaled features to a PyTorch tensor
inputs = torch.tensor(scaled_features, dtype=torch.float32)

# Define the remaining parts of the network (e.g., architecture, loss function, optimizer) and train the model
# ...

# Example usage:
# Forward pass
outputs = model(inputs)

# ...

In [ ]:


# Define the network architecture
class FullyConnectedNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FullyConnectedNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

# Define the hyperparameters
input_size = 784  # Input size for MNIST dataset (28x28 = 784)
hidden_size = 256  # Number of units in the hidden layer
num_classes = 10  # Number of output classes for MNIST (0-9 digits)
learning_rate = 0.001

# Create an instance of the model
model = FullyConnectedNet(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Example usage:
# Forward pass
inputs = torch.randn(100, input_size)  # Random input data of size (batch_size, input_size)
outputs = model(inputs)

# Backward and optimize
labels = torch.randint(num_classes, (100,))  # Random labels of size (batch_size,)
loss = criterion(outputs, labels)
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
# # Define the parameter grid for tuning
# param_grid = {
#     'classifier__n_estimators': [100, 200, 300, 400],
#     'classifier__max_depth': [3, 5, 7, 9, 11],
#     'classifier__min_samples_split': [2, 5, 7],
# }

# # Create the F1 scorer
# scorer = make_scorer(f1_score)
# # Perform grid search cross-validation
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy',refit=True,return_train_score=True,
#                             verbose=1)
# grid_search.fit(X_train, y_train)

# # Get the best model and its performance
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_val)
# report = classification_report(y_val, y_pred)
# print("Best Model Performance:")
# print(report)

# # Get the best hyperparameters
# best_params = grid_search.best_params_
# print("Best Hyperparameters:")
# print(best_params)

In [ ]:
svd = TruncatedSVD(n_components=150)  # Specify the number of components you want to keep
X_svd = svd.fit_transform(X_train_vec)
X_svd_val = svd.transform(X_val_vec)

In [ ]:
lr_svd_clf = LogisticRegression(random_state=42).fit(X_svd, y_train)
lr_preds = lr_svd_clf.predict(X_svd_val)

accuracy = accuracy_score(y_val, lr_preds)
precision = precision_score(y_val, lr_preds)
recall = recall_score(y_val, lr_preds)
f1 = f1_score(y_val, lr_preds)
roc_auc = roc_auc_score(y_val, lr_preds)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')

In [ ]:
X_train.head()

In [ ]:
features = X_train.drop(['original_text','clean_text','stem_text','pos_counts'],axis=1)
features = features.columns.tolist()

In [ ]:
#logistic regression on both tf-idf and features
X_additional = X_train[features].values

# # Concatenate the arrays together
X_train_arr = np.concatenate((X_svd, X_additional), axis=1)

In [ ]:
X_additional_val = X_val[features].values

# # Concatenate the arrays together
X_val_arr = np.concatenate((X_svd_val, X_additional_val), axis=1)

In [ ]:
#create pipeline with .yaml for easy git changes

In [ ]:
lr_arr_clf = LogisticRegression(random_state=42).fit(X_train_arr, y_train)
lr_preds = lr_arr_clf.predict(X_val_arr)

accuracy = accuracy_score(y_val, lr_preds)
precision = precision_score(y_val, lr_preds)
recall = recall_score(y_val, lr_preds)
f1 = f1_score(y_val, lr_preds)
roc_auc = roc_auc_score(y_val, lr_preds)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')